# Projet IA

## Introduction

L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.

La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :

- Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.

- Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.

- Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.

***Spécialistes de l'analyse de données, nous allons déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.***

## Création des chemins d'environnements et analyse de données


### Gestion Execution et images

In [1]:
# imports
import numpy as np
import os

from numpy.random import default_rng
# stabilité du notebook d'une exécution à l'autre
random=default_rng(42) 

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ProjetIA"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID) # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Import des CSV

In [4]:
import pandas as pd
import os

EMPLOYEE_SURVEY_DATA_PATH = os.path.join("data")
GENERAL_DATA_PATH = os.path.join("data")
MANAGER_SURVEY_DATA_PATH = os.path.join("data")
IN_TIME_PATH = os.path.join("data")
OUT_TIME_PATH = os.path.join("data")

def employee_survey_data(employee_path=EMPLOYEE_SURVEY_DATA_PATH):
    csv_path = os.path.join(employee_path, "employee_survey_data.csv")
    return pd.read_csv(csv_path)

def general_data(data_path=GENERAL_DATA_PATH):
    csv_path = os.path.join(data_path, "general_data.csv")
    return pd.read_csv(csv_path) 

def manager_survey_data(manager_survey_path=MANAGER_SURVEY_DATA_PATH):
    csv_path = os.path.join(manager_survey_path, "manager_survey_data.csv")
    return pd.read_csv(csv_path) 

def out_time_data(out_time_path=OUT_TIME_PATH):
    csv_path = os.path.join(out_time_path, "out_time.csv")
    return pd.read_csv(csv_path) 

def in_time_data(in_time_path=IN_TIME_PATH):
    csv_path = os.path.join(in_time_path, "in_time.csv")
    return pd.read_csv(csv_path) 

### Tableau CSV en mémoire

In [5]:
employee = employee_survey_data()
general = general_data()
manager = manager_survey_data()
outTime = out_time_data()
inTime = in_time_data()

### Preparation des heures d'arrivés et de sorties avec le CSV inTime et OutTime

In [6]:
inTimeCpy = inTime.copy()
inTimeCpy = inTimeCpy.fillna(0)
outTimeCpy = outTime.copy()
outTimeCpy = outTimeCpy.fillna(0)

for k in range(2,len(inTimeCpy.columns)):
    inTimeCpy[inTimeCpy.columns[k]] = pd.to_datetime(inTimeCpy[inTimeCpy.columns[k]], format='%Y-%m-%d %H:%M:%S')
for k in range(2,len(outTimeCpy.columns)):
    outTimeCpy[outTimeCpy.columns[k]] = pd.to_datetime(outTimeCpy[outTimeCpy.columns[k]], format='%Y-%m-%d %H:%M:%S')

deltaTime = outTimeCpy-inTimeCpy

deltaTime.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,0,0.0,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,0 days 07:15:46,0 days 07:49:52,...,0 days 00:00:00,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,0 days,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49
1,0,0.0,0 days 08:06:33,0 days 07:27:17,0 days 00:00:00,0 days 07:23:49,0 days 07:25:00,0 days 07:09:03,0 days 07:36:41,0 days 07:16:44,...,0 days 07:54:11,0 days 07:45:14,0 days 07:42:44,0 days 07:26:08,0 days 00:00:00,0 days,0 days 07:36:53,0 days 07:58:57,0 days 07:59:10,0 days 08:13:38
2,0,0.0,0 days 06:41:33,0 days 07:15:56,0 days 06:24:19,0 days 06:45:54,0 days 07:20:42,0 days 06:51:41,0 days 07:25:07,0 days 06:59:59,...,0 days 06:47:09,0 days 07:09:49,0 days 06:48:06,0 days 06:43:49,0 days 06:50:59,0 days,0 days 07:01:26,0 days 07:26:20,0 days 07:32:20,0 days 06:47:11
3,0,0.0,0 days 07:20:18,0 days 07:17:31,0 days 06:56:35,0 days 06:55:10,0 days 06:51:03,0 days 07:11:35,0 days 06:59:55,0 days 07:18:23,...,0 days 07:37:45,0 days 06:50:48,0 days 07:19:35,0 days 07:24:49,0 days 07:05:06,0 days,0 days 07:26:50,0 days 07:25:00,0 days 07:21:59,0 days 07:07:59
4,0,0.0,0 days 08:03:20,0 days 07:59:17,0 days 07:40:57,0 days 07:48:22,0 days 07:39:44,0 days 07:43:18,0 days 08:21:54,0 days 08:15:26,...,0 days 07:54:13,0 days 07:39:54,0 days 07:57:27,0 days 07:47:13,0 days 08:14:58,0 days,0 days 07:39:44,0 days 08:16:07,0 days 07:57:12,0 days 08:01:05


### Calcul de la moyenne des temps de travail par employés (MeanWorkingTime)

In [7]:
meanWorkingTime = pd.DataFrame(data={'EmployeeID': [], 'MeanWorkingTime': []})

for row in range(len(deltaTime)):
    tempRow={'EmployeeID':0, 'MeanWorkingTime':0}
    mean = 0
    for col in range(1,len(deltaTime.columns)):
        tempRow['EmployeeID'] = int(inTime[inTime.columns[0]][row])
        if(deltaTime[deltaTime.columns[col]][row] != 0):
            if(mean != 0):
                mean = (mean+deltaTime[deltaTime.columns[col]][row])/2
            else:
                mean = deltaTime[deltaTime.columns[col]][row]
    tempRow['MeanWorkingTime'] = mean.total_seconds()
    meanWorkingTime = meanWorkingTime.append(tempRow, ignore_index=True)

meanWorkingTime.head()

,EmployeeID,MeanWorkingTime
0,1.0,25562.179418
1,2.0,27729.614998
2,3.0,24697.310763
3,4.0,25293.129258
4,5.0,27940.867653


### Fusion de toute les tables a l'aide du champs EmployeeID

Nous rettirons également les champs :
- EmployeeID
- EmployeeCount
- StandardHours

Ces champs ne nous apporte aucune valeurs étant continue ou des valeurs d'ID.

In [18]:
merge= employee.merge( general, on='EmployeeID')
merge2 = merge.merge(manager, on='EmployeeID')
merge3 = merge2.merge(meanWorkingTime, on='EmployeeID')
workingMerge = merge3
# remove unsuable column because all same value
colToDel = ["EmployeeID", "EmployeeCount", "StandardHours", "Over18"]
workingMerge = workingMerge.drop(colToDel, axis=1)
workingMerge.Attrition.replace(to_replace=dict(Yes=1, No=0), inplace=True)
#Drop rows with Nan
workingMerge.dropna(inplace=True)

workingMerge.to_csv('Data/DataMerge.CSV')


### Création de la pipline

In [24]:
from sklearn.preprocessing import OrdinalEncoder
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


DATA_MERGE_PATH = os.path.join("data")
def DataMerge(data_merge_path=DATA_MERGE_PATH):
    csv_path = os.path.join(data_merge_path, "DataMerge.csv")
    return pd.read_csv(csv_path) 

X = DataMerge()


#remplace gender male=1 / female=0
X.Gender.replace(to_replace=dict(Male=1, Female=0), inplace=True)
 
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
 
X_num = X.select_dtypes(include=[np.number])
 
num_attribs = list(X_num)
cat_attribs = ["MaritalStatus", "JobRole", "BusinessTravel", "Department", "EducationField"]
 
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OrdinalEncoder(), cat_attribs),
    ])
 
X_prepared = full_pipeline.fit_transform(X)


#Visualisation
pd.set_option('display.max_columns', None)
X_prepared

array([[-1.73800917,  0.25240281,  1.15735878, ...,  2.        ,
         2.        ,  1.        ],
       [-1.73722299,  0.25240281, -0.65794042, ...,  1.        ,
         1.        ,  1.        ],
       [-1.73643681, -0.66194603, -0.65794042, ...,  1.        ,
         1.        ,  4.        ],
       ...,
       [ 1.72590965,  1.16675164,  1.15735878, ...,  2.        ,
         1.        ,  3.        ],
       [ 1.72669583, -1.57629486,  0.24970918, ...,  2.        ,
         1.        ,  1.        ],
       [ 1.72748201,  1.16675164, -1.56559001, ...,  2.        ,
         2.        ,  3.        ]])

## Visualisation des champs

## Table visualisation Info et histogramme 

In [27]:
X.head()

,Unnamed: 0,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating,MeanWorkingTime
0,0,3.0,4.0,2.0,51,0,Travel_Rarely,Sales,6,2,Life Sciences,0,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3,3,25562.179418
1,1,3.0,2.0,4.0,31,1,Travel_Frequently,Research & Development,10,1,Life Sciences,0,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,2,4,27729.614998
2,2,2.0,2.0,1.0,32,0,Travel_Frequently,Research & Development,17,4,Other,1,4,Sales Executive,Married,193280,1.0,15,3,5.0,2,5,0,3,3,3,24697.310763
3,3,4.0,4.0,3.0,38,0,Non-Travel,Research & Development,2,5,Life Sciences,1,3,Human Resources,Married,83210,3.0,11,3,13.0,5,8,7,5,2,3,25293.129258
4,4,4.0,1.0,3.0,32,0,Travel_Rarely,Research & Development,10,1,Medical,1,1,Sales Executive,Single,23420,4.0,12,2,9.0,2,6,0,4,3,3,27940.867653


In [ ]:
#graph hist
X.hist(bins=50, figsize=(20,15))
plt.show()

In [22]:
#valeur dans un champs
X["MeanWorkingTime"].value_counts()

22841.840687    1
25730.650937    1
21734.986085    1
28989.641179    1
22344.443794    1
               ..
25293.415928    1
20473.254085    1
27688.348704    1
26930.369825    1
25041.098071    1
Name: MeanWorkingTime, Length: 4300, dtype: int64

## Dendogramme

In [10]:
# plt.figure(figsize=(10, 7))
# dendrogram(linkage(X_prepared, method='ward'), orientation='top', distance_sort='descending', show_leaf_counts=True)
# plt.show()

In [11]:
# cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
# cluster.fit_predict(X)
# plt.figure(figsize=(10, 7))
# plt.scatter(X['MeanWorkingTime'],X['MonthlyIncome'], c=cluster.labels_, cmap='rainbow')
# plt.show()

In [12]:
# # Test de la corrélation entre le temps de travail et les départs
# plt.scatter(X['MeanWorkingTime'],X['StandardHours'], c=cluster.labels_, cmap='rainbow')
# plt.show()
# plt.figure(figsize=(10, 7))
# plt.scatter(X['MeanWorkingTime'],X['StandardHours'], c=X['Attrition'], cmap='rainbow')
# plt.show()

# Création d'un jeu de test
## Jeu de test aléatoire


In [13]:
from sklearn.model_selection import train_test_split
X_full_set, X_test_set = train_test_split(X, test_size=0.8, random_state=42) #SOLUTION
X_test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 1815 to 2982
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               3440 non-null   int64  
 1   EnvironmentSatisfaction  3440 non-null   float64
 2   JobSatisfaction          3440 non-null   float64
 3   WorkLifeBalance          3440 non-null   float64
 4   Age                      3440 non-null   int64  
 5   Attrition                3440 non-null   int64  
 6   BusinessTravel           3440 non-null   object 
 7   Department               3440 non-null   object 
 8   DistanceFromHome         3440 non-null   int64  
 9   Education                3440 non-null   int64  
 10  EducationField           3440 non-null   object 
 11  EmployeeCount            3440 non-null   int64  
 12  Gender                   3440 non-null   object 
 13  JobLevel                 3440 non-null   int64  
 14  JobRole              

In [14]:
X = X_test_set.drop("Attrition", axis=1)
X_labels = X_test_set["Attrition"].copy()

# Classification Ascendante Hiérarchique et dendogramme


# Regression lineaire

In [16]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_prepared, X_labels)

LinearRegression()

## Selection de données aléatoires

In [21]:
# On applique le full_pipeline sur quelques instances :
some_data = X.iloc[:5]
some_labels = X_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

# Et on effectue la prédiction :
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels)) # vraies valeurs

Predictions: [0.27292473 0.21064469 0.23736978 0.19941782 0.39551045]
Labels: [0, 1, 0, 1, 0]


## Evaluation du modele

In [23]:
from sklearn.metrics import mean_squared_error
from math import *
X_predictions = lin_reg.predict(X_prepared)
lin_rmse = sqrt(mean_squared_error(X_labels, X_predictions ))
lin_rmse

0.34372755316204046

# Arbre de décision

In [24]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_prepared, X_labels)
X_predictions = tree_reg.predict(X_prepared)
tree_mse = mean_squared_error(X_labels, X_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

0 => overfitting